In [33]:
from IPython.display import display, HTML, clear_output
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, Layout, HBox, VBox
import pandas as pd
import time
import re
import os
import sys
import requests
import subprocess
import numpy as np
data_dir = './data/experiments'
data_files = [' '] + os.listdir(data_dir)

# Widget styles
default = (data_files[0], ' ')
style = {'description_width': 'initial'}
features_layout = {'width': 'max-content','height':'200px'}

# Drop Down menues
file_dd=widgets.SelectMultiple(options=data_files,value=default,
    description='File:',style=style,layout=features_layout)

# Run buttons
button = widgets.Button(description = "Run",style=style)
button.style.button_color = 'lightgreen'

url = 'http://127.0.0.01:6002/cluster_analysis/api/v0.1/clustering'
def run_service(b):
    file = file_dd.value[0]
    if file == ' ':
        print('No file selected')
    else:
        data_path = os.path.join(data_dir, file)
        files = {'file': open(data_path, 'rb')}
        response = requests.post(url, files=files)
        response = response.json()
        np.save('response.npy', response)

out = widgets.Output() 
#@out.capture(clear_output=True)
def print_results(b):
    #clear_output()
    cluster_key = clusters_keys_dd.value[0]
    if cluster_key == ' ':
        print('No key selected')
    else:
        response = np.load('response.npy', allow_pickle=True)[()]
        clusters_keys = list(response.keys())
        sep_length = len(cluster_key)+5 
        print(sep_length*'=')
        print('key:',cluster_key)
        print(sep_length*'-')
        names = list(response[cluster_key])
        for name in names: print(name)

In [34]:
# Dashboard
output = widgets.Output()
button.on_click(run_service)
display(HTML('<h1 style="color:magenta">Cluster Activities</h1>\
              <p style="color:blue">Use to following menus to submit a file for analysis:</p>\
                 <ul>\
                  <li style="color:magenta">File</li>\
                  <li style="color:magenta">Tune Metrics Weights</li>\
                  <li style="color:magenta">Another Field</li>\
                </ul>'))
HBox(children=[file_dd, button])

In [31]:
response = np.load('response.npy', allow_pickle=True)[()]
clusters_keys = [' '] + list(response.keys())
default = (clusters_keys[0], ' ')
clusters_keys_dd=widgets.SelectMultiple(options=clusters_keys, value=default,
    description='Clusters Keys:',style=style,layout=features_layout)

# Run buttons
button = widgets.Button(description = "Run",style=style)
button.style.button_color = 'lightgreen'

output = widgets.Output()
button.on_click(print_results)
HBox(children=[clusters_keys_dd, button])

In [32]:
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')